# Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import log10,sqrt
from glob import glob
import cv2
import os
import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Data Loading

In [12]:
batch_size = 16

In [13]:
def data_preprocess2(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(images=img, size=[400, 400])
    img = img / 255.0
    return img
def data_preprocess1(images):
    img = tf.data.Dataset.from_tensor_slices((images))
    img = img.map(data_preprocess2, num_parallel_calls=tf.data.AUTOTUNE)
    img = img.batch(batch_size, drop_remainder=True)
    return img

In [4]:
train_images = sorted(glob("/content/drive/MyDrive/lol_dataset/our485/low/*"))[:420]
val_images = sorted(glob("/content/drive/MyDrive/lol_dataset/our485/low/*"))[420:]
test_images = sorted(glob("/content/drive/MyDrive/lol_dataset/eval15/low/*"))


train_dataset = data_preprocess1(train_images)
val_dataset = data_preprocess1(val_images)
batch_size=1
test_dataset = data_preprocess1(test_images)

# Model

In [5]:
def build_dce_net():
    input = keras.Input(shape=[None, None, 3])
    c0 = layers.Conv2D(16, (3, 3), strides=(1, 1), activation="relu", padding="same")(input)
    c1 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(c0)
    c2 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(c1)
    c3 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(c2)
    c4 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(c3)
    con1 = layers.Concatenate(axis=-1)([c4, c3])
    c5 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(con1)
    con2 = layers.Concatenate(axis=-1)([c5, c2])
    c6 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(con2)
    con3 = layers.Concatenate(axis=-1)([c6, c1])
    output = layers.Conv2D(24, (3, 3), strides=(1, 1), activation="tanh", padding="same")(con3)
    return keras.Model(inputs=input, outputs=output)

# Losses

In [6]:
def ConstancyLoss(image_tensor):
    # Compute the mean RGB values across the height and width dimensions
    mean_colors = tf.reduce_mean(image_tensor, axis=(1, 2), keepdims=True)

    # Extract the mean values for each channel
    mean_r, mean_g, mean_b = (
        mean_colors[:, :, :, 0],
        mean_colors[:, :, :, 1],
        mean_colors[:, :, :, 2],
    )

    # Calculate the squared differences between the mean values of each pair of channels
    diff_rg = tf.square(mean_r - mean_g)
    diff_rb = tf.square(mean_r - mean_b)
    diff_gb = tf.square(mean_b - mean_g)

    # Return the combined loss as the square root of the sum of the squared differences
    return tf.sqrt(diff_rg + diff_rb + diff_gb)

In [7]:
def ExposureLoss(image_tensor, target_mean_value=0.6):
    # Compute the mean intensity value across the RGB channels
    mean_intensity = tf.reduce_mean(image_tensor, axis=3, keepdims=True)

    # Perform average pooling with a kernel size and stride of 16
    pooled_mean_intensity = tf.nn.avg_pool2d(mean_intensity, ksize=16, strides=16, padding="VALID")

    # Calculate the exposure loss as the mean squared difference from the target mean value
    ExposureLoss_value = tf.reduce_mean(tf.square(pooled_mean_intensity - target_mean_value))

    return ExposureLoss_value

In [8]:
def SmoothnessLoss(image_tensor):
    batch_size = tf.shape(image_tensor)[0]
    height = tf.shape(image_tensor)[1]
    width = tf.shape(image_tensor)[2]
    channels = tf.shape(image_tensor)[3]

    # Calculate the total number of horizontal and vertical differences
    total_horizontal = (width - 1) * channels
    total_vertical = width * (channels - 1)

    # Compute the horizontal and vertical total variation
    horizontal_variation = tf.reduce_sum(tf.square(image_tensor[:, 1:, :, :] - image_tensor[:, :-1, :, :]))
    vertical_variation = tf.reduce_sum(tf.square(image_tensor[:, :, 1:, :] - image_tensor[:, :, :-1, :]))

    # Cast to float32 for division
    total_horizontal = tf.cast(total_horizontal, dtype=tf.float32)
    total_vertical = tf.cast(total_vertical, dtype=tf.float32)
    batch_size = tf.cast(batch_size, dtype=tf.float32)

    # Return the average smoothness loss per batch
    return 2 * (horizontal_variation / total_horizontal + vertical_variation / total_vertical) / batch_size

In [9]:
class SpatialConsistencyLoss(keras.losses.Loss):
    def __init__(self, **kwargs):
        super().__init__(reduction="none")
        self.kernels = {
            "left": tf.constant([[[[0, 0, 0]], [[-1, 1, 0]], [[0, 0, 0]]]], dtype=tf.float32),
            "right": tf.constant([[[[0, 0, 0]], [[0, 1, -1]], [[0, 0, 0]]]], dtype=tf.float32),
            "up": tf.constant([[[[0, -1, 0]], [[0, 1, 0]], [[0, 0, 0]]]], dtype=tf.float32),
            "down": tf.constant([[[[0, 0, 0]], [[0, 1, 0]], [[0, -1, 0]]]], dtype=tf.float32)
        }

    def call(self, true, pred):
        mean_true = tf.reduce_mean(true, axis=3, keepdims=True)
        mean_pred = tf.reduce_mean(pred, axis=3, keepdims=True)

        pool_true = tf.nn.avg_pool2d(mean_true, ksize=4, strides=4, padding="VALID")
        pool_pred = tf.nn.avg_pool2d(mean_pred, ksize=4, strides=4, padding="VALID")

        diffs = []
        for direction, kernel in self.kernels.items():
            diff_true = tf.nn.conv2d(pool_true, kernel, strides=[1, 1, 1, 1], padding="SAME")
            diff_pred = tf.nn.conv2d(pool_pred, kernel, strides=[1, 1, 1, 1], padding="SAME")
            diffs.append(tf.square(diff_true - diff_pred))

        return tf.add_n(diffs)

In [10]:
class ZeroDCE(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dce_model = build_dce_net()

    def compile(self, learning_rate, **kwargs):
        super().compile(**kwargs)
        self.optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        self.spatial_constancy_loss = SpatialConsistencyLoss(reduction="none")
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.SmoothnessLoss_tracker = keras.metrics.Mean(
            name="SmoothnessLoss"
        )
        self.spatial_constancy_loss_tracker = keras.metrics.Mean(
            name="spatial_constancy_loss"
        )
        self.ConstancyLoss_tracker = keras.metrics.Mean(
            name="ConstancyLoss"
        )
        self.ExposureLoss_tracker = keras.metrics.Mean(name="ExposureLoss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.SmoothnessLoss_tracker,
            self.spatial_constancy_loss_tracker,
            self.ConstancyLoss_tracker,
            self.ExposureLoss_tracker,
        ]

    def get_enhanced_image(self, data, output):
        r1 = output[:, :, :, :3]
        r2 = output[:, :, :, 3:6]
        r3 = output[:, :, :, 6:9]
        r4 = output[:, :, :, 9:12]
        r5 = output[:, :, :, 12:15]
        r6 = output[:, :, :, 15:18]
        r7 = output[:, :, :, 18:21]
        r8 = output[:, :, :, 21:24]
        x = data + r1 * (tf.square(data) - data)
        x = x + r2 * (tf.square(x) - x)
        x = x + r3 * (tf.square(x) - x)
        enhanced_image = x + r4 * (tf.square(x) - x)
        x = enhanced_image + r5 * (tf.square(enhanced_image) - enhanced_image)
        x = x + r6 * (tf.square(x) - x)
        x = x + r7 * (tf.square(x) - x)
        enhanced_image = x + r8 * (tf.square(x) - x)
        return enhanced_image

    def call(self, data):
        dce_net_output = self.dce_model(data)
        return self.get_enhanced_image(data, dce_net_output)

    def compute_losses(self, data, output):
        enhanced_image = self.get_enhanced_image(data, output)
        loss_illumination = 200 * SmoothnessLoss(output)
        loss_spatial_constancy = tf.reduce_mean(
            self.spatial_constancy_loss(enhanced_image, data)
        )
        loss_color_constancy = 5 * tf.reduce_mean(ConstancyLoss(enhanced_image))
        loss_exposure = 10 * tf.reduce_mean(ExposureLoss(enhanced_image))
        total_loss = (
            loss_illumination
            + loss_spatial_constancy
            + loss_color_constancy
            + loss_exposure
        )

        return {
            "total_loss": total_loss,
            "SmoothnessLoss": loss_illumination,
            "spatial_constancy_loss": loss_spatial_constancy,
            "ConstancyLoss": loss_color_constancy,
            "ExposureLoss": loss_exposure,
        }

    def train_step(self, data):
        with tf.GradientTape() as tape:
            output = self.dce_model(data)
            losses = self.compute_losses(data, output)

        gradients = tape.gradient(
            losses["total_loss"], self.dce_model.trainable_weights
        )
        self.optimizer.apply_gradients(zip(gradients, self.dce_model.trainable_weights))

        self.total_loss_tracker.update_state(losses["total_loss"])
        self.SmoothnessLoss_tracker.update_state(
            losses["SmoothnessLoss"]
        )
        self.spatial_constancy_loss_tracker.update_state(
            losses["spatial_constancy_loss"]
        )
        self.ConstancyLoss_tracker.update_state(losses["ConstancyLoss"])
        self.ExposureLoss_tracker.update_state(losses["ExposureLoss"])

        return {metric.name: metric.result() for metric in self.metrics}

    def test_step(self, data):
        output = self.dce_model(data)
        losses = self.compute_losses(data, output)

        self.total_loss_tracker.update_state(losses["total_loss"])
        self.SmoothnessLoss_tracker.update_state(
            losses["SmoothnessLoss"]
        )
        self.spatial_constancy_loss_tracker.update_state(
            losses["spatial_constancy_loss"]
        )
        self.ConstancyLoss_tracker.update_state(losses["ConstancyLoss"])
        self.ExposureLoss_tracker.update_state(losses["ExposureLoss"])

        return {metric.name: metric.result() for metric in self.metrics}

    def save_weights(self, filepath, overwrite=True, save_format=None, options=None):

        self.dce_model.save_weights(
            filepath,
            overwrite=overwrite,
            save_format=save_format,
            options=options,
        )

    def load_weights(self, filepath, by_name=False, skip_mismatch=False, options=None):

        self.dce_model.load_weights(
            filepath=filepath,
            by_name=by_name,
            skip_mismatch=skip_mismatch,
            options=options,
        )

In [ ]:
model = ZeroDCE()
model.compile(learning_rate=1e-4)
history = model.fit(train_dataset, validation_data=val_dataset, epochs=100)
model.save('/content/drive/MyDrive/lol_dataset/sphgetti_net.h5')

Epoch 1/100
21/21 [==============================] - 22s 813ms/step - total_loss: 4.1571 - SmoothnessLoss: 1.1280 - spatial_constancy_loss: 1.3558e-05 - ConstancyLoss: 0.1024 - ExposureLoss: 2.9267 - val_total_loss: 4.1833 - val_SmoothnessLoss: 1.0368 - val_spatial_constancy_loss: 8.6749e-06 - val_ConstancyLoss: 0.1050 - val_ExposureLoss: 3.0415
Epoch 2/100
21/21 [==============================] - 16s 734ms/step - total_loss: 3.5808 - SmoothnessLoss: 0.5570 - spatial_constancy_loss: 1.8078e-05 - ConstancyLoss: 0.1018 - ExposureLoss: 2.9219 - val_total_loss: 3.7405 - val_SmoothnessLoss: 0.5974 - val_spatial_constancy_loss: 1.1162e-05 - val_ConstancyLoss: 0.1059 - val_ExposureLoss: 3.0373
Epoch 3/100
21/21 [==============================] - 15s 712ms/step - total_loss: 3.3464 - SmoothnessLoss: 0.3272 - spatial_constancy_loss: 2.1970e-05 - ConstancyLoss: 0.1024 - ExposureLoss: 2.9167 - val_total_loss: 3.5288 - val_SmoothnessLoss: 0.3900 - val_spatial_constancy_loss: 1.5377e-05 - val_Const

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/lol_dataset/sphgetti_net.h5')

In [ ]:
pred=model.predict(test_dataset)

In [ ]:
y_test_images = sorted(glob("/content/drive/MyDrive/lol_dataset/eval15/high/*"))
y_test=[]
for file in y_test_images:
  img = tf.io.read_file(file)
  img = tf.image.decode_png(img, channels=3)
  img = tf.image.resize(images=img, size=[400, 400])
  img = img / 255.0
  y_test.append(img)

In [ ]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):
        return 100
    max_pixel = 1.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [ ]:
y_test=np.array(y_test)

In [ ]:
PSNR(y_test,pred)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test,pred)

In [ ]:
def mse(image01,image02):
    error= np.sum((image01.astype("float")- image02.astype("float"))**2)
    error= error/float(image01.shape[0]*image02.shape[1])
    return error

In [ ]:
def mae(image01,image02):
    error= np.sum(abs(image01.astype("float")- image02.astype("float")))
    error= error/float(image01.shape[0]*image02.shape[1])
    return error

In [ ]:
mse_err=[]
mae_err=[]
for i in range(len(y_test)):
  mse_err.append(mse(y_test[i],pred[i]))
  mae_err.append(mae(y_test[i],pred[i]))
print(np.mean(mse_err))
print(np.mean(mae_err))